# Docker

### Purging all dangling images, container, volumes and networks

`docker system prune` _Prune means to remove unwanted stuff_

`docker system prune -a` _To remove all_

## ----------------------------------- CONTAINERS -----------------------------------

### List all containers
`docker ps -a`

`docker ps -aq` [q: only ids]

### Stop all running containers
`docker stop $(docker ps -aq)`

### Remove all containers
`docker rm $(docker ps -aq)`

### Run and remove
`docker run -rm image_name`

### Remove only exited containers
`docker rm $(docker ps -aq -f status=exited)`

### Remove containers according to pattern
`docker ps -a | grep "pattern" | awk '{print $1}' | xargs docker rm`
> awk and xargs to supply the ID to docker rm

## ----------------------------------- IMAGES -----------------------------------

### List images
`docker images`

### Remove all images
`docker rmi $(docker images -aq)` _q: for id_

### List dangling images
`docker images -f dangling=true`

### Remove dangline images
`docker image prune`

### Remove images according to pattern
`docker images -a | grep "pattern" | awk '{print $3}' | xargs docker rmi`


## ----------------------------------- VOLUMES -----------------------------------

### List volumes
`docker volume ls`

### Remove volume
`docker volume rm vol_name`

### List dangling volumes
`docker volume ls -f dangling=true`

### Remove dangling volumes
`docker volume prune`

## Remove container and its volume
`docker rm -v container_name`

__SUMMARY__
> docker system prune

> docker rmi

> docker rm

> docker volume rm

-----------


### To run bash on docker image
```bash
# Assuming an Ubuntu Docker image
$ docker run -it --name <container_name> <image> /bin/bash
```

> This won't work if your image has a defined ENTRYPOINT. For these cases use: `docker run -it --entrypoint /bin/bash <image>`

### To go inside container running in detached state
`docker exec -it cc73eb6d6f75 bash`

------------

## Theory on Docker and Docker Layer Caching

This is where the paradigm shift comes into place: software is no longer packaged as a platform-specific binary artifact (jar, dll, tgz) but as a fully fledged virtual environment in the form of Docker images. This means that developers can run the code locally exactly as it is run on dev, test or prod environments. The operations teams have only Docker images to deal with, with much less need to understand the inner workings of the specific platform they are deploying.

Every time you change or update the application code, you need to build a new version of the image that can be used for deployment. Even though you’d typically only change the application code, the entire image needs to be built from scratch — including all dependencies. To help with the efficiency of the typical development process and shorten the feedback loop cycle, Docker has introduced the concept of layer caching.

## Theory on Docker Compose and Buildkit

We can use `docker build` and `docker run` to build and test/run our applications. But if we have multiple docker files for e.g for client app and one for server then to switch back and forth and build and run these can be cumbersome. To solve this, we have `docker-compose` that can build and run with just one command.

With the help of Buildkit (still experimental) we can use caching layer service of docker build into docker-compose. When we use docker build and then docker compose the docker compose builds the dockerfile from scratch and doesn't use the cache of docker build that we ran first time. With the help of buildkit command docker-compose will re-use the cache layer.

Note: when we use builtkit for the first time, it'll create it's own layer storage strategy. This means it'll build the docker image from scratch but then after that when we use docker-compose, it will use the buildkit cache storage and would not build from scratch.

Brief introduction [5mins read]: https://medium.com/better-programming/sharing-cached-layer-between-docker-and-docker-compose-builds-c2e5f751cee4


# Docker Contd.

## Share data between docker container and host

## Bind mount volumes

Docker containers are emphemeral meaning any data created inside the container is only available in that container and only while the container is running.

__Scenario__:

Let's say we want to run nginx container and keep the permanent copy of log files generated during it's run for later analysis. Nginx log files are generated at /var/log/nginx by default and it is not accessible from host system.

__Step 1: Bindmount Volume__:

`docker run --name=nginx -d -v ~/nginx_logs:/var/logs/nginx -p 5000:80 nginx`

* `--name=nginx` names the container so that we can refer it easily
* `-d` run the container in detached state i.e in background so that we can have access to the terminal from where we are running the docker run command
* `-v ~/nginx_logs:/var/logs/nginx` bindmounts volume that links `/var/logs/nginx` directory from inside the container to `~/nginx_logs` directory of the host. Docker uses __`:`__ to split host path with container path and __host path always comes first__. 
* `-p 5000:80` __port forwarding__. This flag maps container's port 80 to port 5000 of the host machine.
* `nginx` name of the image

> The -v flag is very flexible. It can bindmount or name a volume with just a slight adjustment in syntax. If the first argument begins with a / or ~/, you’re creating a bindmount. Remove that, and you’re naming the volume.
For more details: https://www.digitalocean.com/community/tutorials/how-to-share-data-between-docker-containers

__Step 2: Access Data on Host__:
Just go to the directory `~/nginx_logs` and see the log file.
> If you make any changes to the `~/nginx_logs` folder, you’ll be able to see them from inside the Docker container in real time as well.

> Multiple containers can share the same bind mount.

## Named Volumes

- They are more recent ways of creating volumes and they exist outside the container lifecycle.
- Named volumes support more feature thatn bindmount such as remote cloud storage

```docker
docker create volume my_volume
docker run -v my_volume:/directory/in/container ...
```

# Docker Syntax

EXAMPLE 1:
```python
FROM node:10.9.0

COPY . .

RUN npm install

EXPOSE 8080

CMD npm start

# EXPLANATION:

# FROM it's better to use version rather than latest as the latest will override the current working docker image and if the latest image has problem then it's tricky to roll back. Versioning gives more control

# COPY current directory to working directory in container. Here the working dir of container will inherit from node image.
# ADD is more advanced/powerful than COPY. For e.g it automatically extracts the archive and it has support for URLs. Use COPY by default to avoid any surprises unless you specifically need it.

# RUN will run the shell command in the current workind directory

# EXPOSE tell what port the container should be listening on. It's merely for documentation as it does not actually publish the port number on the host machine. To publish the ports and enable the client to connect to them, we need to use `-p` flag from the command line 

# CMD tells what command to run when someone starts the container. The above cmd tells docker to run node server for our application on start.
```

---

# Bash

1. find [where_to_find] -name [what_to_find]

e.g `find / -name *.whl`

## GetIntoDevOps Notes

### CI

- Emphasises automated testing to ensure the new code changes work as intended and does not break anything (fast feedback)
- Pipeline automation servers (like Jenkins) are used to implement automatic testing

### CD (Continuous Delivery)

- While CI is the act of merging code as fast as possible, CD is the act of shipping changes to production frequently, in small increments
- In practice, the code in main branch should be deployable to production at all times

### CD (Continuous Deployment)

- While Delivery makes sure everything in the main branch should be in deployable state, the actual automation of deployment without human intervention is part of this stage.

### A/B Testing

- Introducing two changes in the application and measuring which variant works better

### 


# Journey Questions

1. Basic pipeline running for various repos
2. Use docker
3. Implement bazel caching
4. Implement docker caching